# AcuityMD Challenge

Below is some code that simply loads the data stored in `database.db`, so that you can execute some SQL queries using the pandas library. 

## 1. Find the Top HCPs

### 1.1 Install Libraries

Note, please do not use any additional libraries. Your solution should be completed in SQL.

In [2]:
%pip install pandas
import pandas as pd
import sqlite3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 502 kB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 825 kB/s eta 0:00:01
     |████████████████████████████████| 341 kB 54.0 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### 1.2 Implementation

#### 1.2.1 Check the table names

In [3]:

con = sqlite3.connect('database.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('sqlite_sequence',), ('HealthcareProfessionals',), ('SiteOfCares',), ('ProcedureData',), ('SaleData',)]


#### 1.2.2 Make SQL Query

##### ProcedureData

In [24]:
df = pd.read_sql_query("SELECT hcp, soc, volume FROM ProcedureData LIMIT 10", con)
print(df.head(5))

   hcp  soc  volume
0    0    0      15
1    0   91      15
2    0   73      13
3    0   14      14
4    1   13      73
5    2   78      27
6    2   54      25
7    2   66      18
8    4    9      22
9    4    6       6


In [37]:
df = pd.read_sql_query("SELECT COUNT(DISTINCT hcp) hcp_cnt FROM ProcedureData UNION ALL SELECT COUNT(DISTINCT hcp) hcp_cnt FROM SaleData UNION ALL SELECT COUNT(DISTINCT s.hcp) hcp_cnt FROM SaleData s inner join ProcedureData p ON s.hcp = p.hcp", con)
print(df.head(5))

   hcp_cnt
0      814
1      334
2      334


In [28]:
df = pd.read_sql_query("SELECT DISTINCT hcp,soc, volume as procedure_volume FROM ProcedureData", con)
print(df.head(100))


    hcp  soc  procedure_volume
0     0    0                15
1     0   91                15
2     0   73                13
3     0   14                14
4     1   13                73
..  ...  ...               ...
95   40   99                17
96   41   17                21
97   41   78                39
98   42   74                 6
99   42   78                 5

[100 rows x 3 columns]


##### SaleData

In [26]:
df = pd.read_sql_query("SELECT hcp,soc, volume as sales_volume FROM SaleData LIMIT 10", con)
print(df.head(10))

   hcp  soc  sales_volume
0    0    0            15
1    0   91            15
2    0   73            13
3    1   13            73
4    2   54            25
5    6   31            16
6    6   17            12
7    6   47            19
8   11   40            23
9   13   74             0


#### 1.2.3 Elephant
##### gap between sales and procedures completed by <mark>an HCP<mark>
##### procedure_volume - sales_volume

In [43]:
df = pd.read_sql_query("SELECT DISTINCT p.hcp, SUM(p.volume) total_procedure_volume,SUM(s.volume) total_sales_volume, SUM(p.volume) - SUM(s.volume) gap FROM ProcedureData p LEFT JOIN SaleData s ON s.hcp = p.hcp GROUP BY 1 ORDER BY 4 desc LIMIT 10", con)
print(df.head(50))

   hcp  total_procedure_volume  total_sales_volume  gap
0  877                     224                  36  188
1  627                      99                   0   99
2  946                      62                   4   58
3  317                      96                  40   56
4   55                     142                  92   50
5  189                      52                   4   48
6  366                      59                  12   47
7  573                      46                   0   46
8  214                      44                   0   44
9  179                      48                   4   44


### 1.3 Notes

#### 1.3.1 Assumptions:

* **Time Consistency**: To ensure an unbiased comparison among HCPs, it is imperative that we capture sales volume and procedure volume data over the same time frame for all HCPs. I assume the duration of time among different HCPs is consistant.

* **Seasonal Influence**: Even when data spans the same duration for each HCP, disparities in timing, such as one HCP's data covering the first quarter and another's spanning the fourth quarter, can introduce bias due to seasonal variations. I assume seasonality doesn't play a role here. 

* **Data Accuracy and Currency**: Data quality can be variable. While sales data is likely to be clean and up-to-date(more likely to be internal data), procedure data involves multiple parties and may suffer from inaccuracies and delays. For the purpose of this analysis, I assume that all data is both accurate and current.

* **Device Usage Exclusivity**: I assume that each HCP exclusively uses the medical devices they purchase. In cases where multiple HCPs exist within an SOC, we presume that devices are not shared among them.

#### 1.3.2 Explanation:

* The Elephant method calculates the gap between sales and procedures completed by an HCP, and it is independent of the SOC. Thus, SOC does not affect and is not included in this calculation.

* There are 814 unique HCPs in the ProcedureData table and 334 unique HCPs in the SaleData table. All HCPs in the SaleData table are also present in the ProcedureData table, suggesting that a LEFT JOIN should be used.

* The results are sorted based on the "gap" and the top 5 results are displayed.

## 2. Written Extension

In this step, we would like you to add the answer to the following questions in this section.
There can be multiple ways to prioritize HCPs to target, for example one might use an "upsell" strategy, which would prioritize HCPs with the most existing sales.

- How would you change your solution to handle returning the "top" HCPs based on multiple strategies, for example both "upsell" and "elephant"? Describe your approach, without writing any code. How would you consider sorting HCPS when using multiple strategies? 

Describe your approach, without writing any code. Given this description, an engineer should be able to implement a solution. Please keep your response under 500 words.

### 2.1 Answer:

I aim to identify the top HCPs by combining the "upsell" and "elephant" methods. Assuming that the upsell method results are precomputed and available in a table for each HCP, here are my proposed sorting methods:

* **scoring system:** Initially, I recommend merging the two tables and introducing a new column named "score." This score can be derived using the formula (a X gap) + (b X most existing sales), with 'a' and 'b' representing coefficients.(appropriate coefficient values are identified during the data exploration phase, and are given to the engineer who is bulding this). Subsequently, the table should be sorted in descending order based on the new score, allowing us to select the top-performing HCPs.

* **ranking:** Another viable approach involves utilizing the row_number() function to establish rankings for each HCP, independently for each strategy. Afterward, we can aggregate the rankings across strategies, sort all HCPs in ascending order based on the cumulative rankings, and choose the top-performing HCPs.

Each of these methods comes with its pros and cons. While both can be employed for more than two strategies, offering versatility in the analysis, here are some key considerations:

Ranking approach offers a swift solution, but it can lead to situations where multiple HCPs share the same rank. For instance, we might have four HCPs with a cumulative ranking of 5. This can be less precise in distinguishing their performance.

On the other hand, the scoring method demands more time, particularly during the data exploration phase, but it yields more dependable results. It provides a more accurate evaluation of HCP performance by considering coefficient values, offering better insights into their effectiveness.